In [1]:
pip install transformers datasets Pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 14.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [2]:
from concurrent.futures import ThreadPoolExecutor
from functools import partial
import io
import urllib.request
import PIL.Image
from datasets import load_dataset
from datasets.utils.file_utils import get_datasets_user_agent

USER_AGENT = get_datasets_user_agent()

def fetch_single_image(image_url, timeout=None, retries=0):
    for _ in range(retries + 1):
        try:
            request = urllib.request.Request(
                image_url,
                data=None,
                headers={"user-agent": USER_AGENT},
            )
            with urllib.request.urlopen(request, timeout=timeout) as req:
                image = PIL.Image.open(io.BytesIO(req.read()))
            break
        except Exception:
            image = None
    return image

def fetch_images(batch, num_threads, timeout=None, retries=0):
    fetch_single_image_with_args = partial(fetch_single_image, timeout=timeout, retries=retries)
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        batch["image"] = list(executor.map(fetch_single_image_with_args, batch["image_url"]))
    return batch


In [3]:
dset = load_dataset("conceptual_captions")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for conceptual_captions contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conceptual_captions
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  

Generating train split:   0%|          | 0/3318333 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/15840 [00:00<?, ? examples/s]

In [4]:
# Load the dataset (if not already loaded)
from datasets import load_dataset

dset = load_dataset("conceptual_captions")

# Select a small subset of the dataset for demonstration
subset = dset['train'].select(range(10))

# Now apply the fetch_images function to just this small subset
num_threads = 20  # Adjust based on your machine's capabilities
subset = subset.map(fetch_images, batched=True, batch_size=10, fn_kwargs={"num_threads": num_threads, "timeout": 10, "retries": 2})


/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for conceptual_captions contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conceptual_captions
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [5]:
from transformers import BlipProcessor, BlipForConditionalGeneration

model_id = "Salesforce/blip-image-captioning-large"
processor = BlipProcessor.from_pretrained(model_id)
model = BlipForConditionalGeneration.from_pretrained(model_id)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

In [6]:
def generate_caption(image, processor, model):
    inputs = processor(images=image, return_tensors="pt")

    output_ids = model.generate(**inputs, max_length=30, num_beams=3).cpu().detach().numpy()
    caption = processor.decode(output_ids[0], skip_special_tokens=True)

    return caption

In [7]:
!pip install rouge-score

import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer

# Download NLTK models (if not already downloaded)
nltk.download('wordnet')

original_captions = []
generated_captions = []


for i in range(len(subset)):
    item = subset[i]
    image = item.get('image', None)  # Use .get() to safely return None if 'image' key doesn't exist
    original_caption = item.get('caption', '').strip()  # Safely get caption, default to empty string and strip
    if image and original_caption:  # Proceed if both image and original caption exist
        generated_caption = generate_caption(image, processor, model)
        print(f"Generated: {generated_caption}\nOriginal: {original_caption}\n")
        original_captions.append(original_caption)
        generated_captions.append(generated_caption)


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=4c37b2f2de46177a9d0184ac61e9e9eca63350d56fbb7ca34cf398268994cde2
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


[nltk_data] Downloading package wordnet to /root/nltk_data...


Generated: there is a lot of buses parked in a parking lot with people walking around
Original: a very typical bus station

Generated: two pictures of a woman in a skirt and a t - shirt on a stage
Original: sierra looked stunning in this top and this skirt while performing with person at their former university

Generated: fireplace in a modern living room with white walls and wood flooring
Original: interior design of modern living room with fireplace in a new house

Generated: a robotic hand holding a knife on a blue background
Original: cybernetic scene isolated on white background .

Generated: there is a diagram of a park with several different plants and animals
Original: the jetty : different types of plants to establish a variety of ecosystems .

Generated: traditional ornamental floral paisley design on a green, purple and yellow background
Original: traditional ornamental floral paisley bandanna .

Generated: hockey player in white and blue uniform on the ice with a hockey st

In [10]:
!pip install rouge-score
from nltk.translate.bleu_score import SmoothingFunction, sentence_bleu, corpus_bleu
from rouge_score import rouge_scorer
from nltk.translate.meteor_score import meteor_score

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeLsum'], use_stemmer=True)

# Variables to store scores
bleu_scores = []
meteor_scores = []
rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeLsum': []}

# Iterate through original and generated captions to compute scores
for orig, gen in zip(original_captions, generated_captions):
    # BLEU score calculation (expecting a list of reference sentences)
    bleu_score = sentence_bleu([orig.split()], gen.split(), smoothing_function=SmoothingFunction().method1)
    bleu_scores.append(bleu_score)

    # METEOR score calculation with tokenized input
    meteor_scores.append(meteor_score([orig.split()], gen.split()))

    # ROUGE scores
    scores = scorer.score(orig, gen)
    rouge_scores['rouge1'].append(scores['rouge1'].fmeasure)
    rouge_scores['rouge2'].append(scores['rouge2'].fmeasure)
    rouge_scores['rougeLsum'].append(scores['rougeLsum'].fmeasure)

# Print individual BLEU scores
print(bleu_scores)

# Calculate average scores for all metrics
avg_bleu = sum(bleu_scores) / len(bleu_scores)
avg_meteor = sum(meteor_scores) / len(meteor_scores)
avg_rouge1 = sum(rouge_scores['rouge1']) / len(rouge_scores['rouge1'])
avg_rouge2 = sum(rouge_scores['rouge2']) / len(rouge_scores['rouge2'])
avg_rougeLsum = sum(rouge_scores['rougeLsum']) / len(rouge_scores['rougeLsum'])

# Output average scores
print(f"Average BLEU Score: {avg_bleu}")
print(f"Average METEOR Score: {avg_meteor}")
print(f"Average ROUGE-1 F Measure: {avg_rouge1}")
print(f"Average ROUGE-2 F Measure: {avg_rouge2}")
print(f"Average ROUGE-Lsum F Measure: {avg_rougeLsum}")

# Tokenize the captions for corpus BLEU calculation
tokenized_originals = [[orig.split()] for orig in original_captions]  # List of list of words for each reference
tokenized_generated = [gen.split() for gen in generated_captions]  # List of words for each hypothesis

# Corpus BLEU score calculation
chencherry = SmoothingFunction()
corpus_bleu_score = corpus_bleu(tokenized_originals, tokenized_generated, smoothing_function=chencherry.method1)
print(f"Corpus BLEU Score: {corpus_bleu_score}")

[0.013217947626377298, 0.01520779505248453, 0.30661487102926754, 0.025098621243978974, 0.020345970436499224, 0.21200626759025185, 0.01428363257865929]
Average BLEU Score: 0.08668215793678838
Average METEOR Score: 0.23842660457016923
Average ROUGE-1 F Measure: 0.2844887955182073
Average ROUGE-2 F Measure: 0.12207792207792208
Average ROUGE-Lsum F Measure: 0.2169887955182073
Corpus BLEU Score: 0.07984325631440924


In [11]:
from transformers import pipeline
image_to_text_pipeline = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/4.61k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/982M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/241 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


In [12]:
original_captions = []
generated_captions = []

for item in subset:
    # Assuming each item in subset contains 'image' (URL) and 'caption'
    image_url = item['image_url']
    original_caption = item['caption']

    # Generate caption using the pipeline
    try:
        generated_caption_result = image_to_text_pipeline(image_url)
        generated_caption = generated_caption_result[0]['generated_text']
        print(f"Original: {original_caption}\nGenerated: {generated_caption}\n")

        original_captions.append(original_caption)
        generated_captions.append(generated_caption)
    except Exception as e:
        print(f"Failed to generate caption for image {image_url}: {e}")


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.


Original: a very typical bus station
Generated: a busy street with several buses and people 

Original: sierra looked stunning in this top and this skirt while performing with person at their former university
Generated: a woman standing next to a microphone with a microphone 

Failed to generate caption for image https://media.gettyimages.com/photos/young-confused-girl-standing-in-front-of-a-wardrobe-picture-id511063329?s=612x612: cannot identify image file <_io.BytesIO object at 0x7de8f5026fc0>
Original: interior design of modern living room with fireplace in a new house
Generated: a fireplace with a fire place in it 

Original: cybernetic scene isolated on white background .
Generated: a motorcycle with a propeller attached to it 

Failed to generate caption for image https://media.gettyimages.com/photos/jayz-attends-the-chicago-bulls-vs-brooklyn-nets-playoff-game-at-the-picture-id167112882: cannot identify image file <_io.BytesIO object at 0x7de78579acf0>
Original: the jetty : diff

In [14]:
from nltk.translate.bleu_score import SmoothingFunction, corpus_bleu
from rouge_score import rouge_scorer
from nltk.translate.meteor_score import single_meteor_score

# Tokenize captions for BLEU calculation
tokenized_originals = [[oc.split()] for oc in original_captions]
tokenized_generated = [gc.split() for gc in generated_captions]

# Compute BLEU score
chencherry = SmoothingFunction()
bleu_score = corpus_bleu(tokenized_originals, tokenized_generated, smoothing_function=chencherry.method1)

# Compute METEOR scores with tokenized inputs
meteor_scores = [meteor_score([oc.split()], gc.split()) for oc, gc in zip(original_captions, generated_captions)]
avg_meteor = sum(meteor_scores) / len(meteor_scores)

# Compute ROUGE scores including ROUGE-2
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = [scorer.score(oc, gc) for oc, gc in zip(original_captions, generated_captions)]
avg_rouge1 = sum(r['rouge1'].fmeasure for r in rouge_scores) / len(rouge_scores)
avg_rouge2 = sum(r['rouge2'].fmeasure for r in rouge_scores) / len(rouge_scores)
avg_rougeL = sum(r['rougeL'].fmeasure for r in rouge_scores) / len(rouge_scores)

print(f"BLEU Score: {bleu_score}")
print(f"Average METEOR Score: {avg_meteor}")
print(f"Average ROUGE-1 F Measure: {avg_rouge1}")
print(f"Average ROUGE-2 F Measure: {avg_rouge2}")
print(f"Average ROUGE-L F Measure: {avg_rougeL}")

BLEU Score: 0.0092564717187886
Average METEOR Score: 0.07437539019121867
Average ROUGE-1 F Measure: 0.14827431827431828
Average ROUGE-2 F Measure: 0.012987012987012988
Average ROUGE-L F Measure: 0.09636955636955637


In [15]:
from transformers import pipeline

# Initialize Pix2Struct image captioning pipeline
pix2struct_caption_pipeline = pipeline("image-to-text", model="google/pix2struct-textcaps-base")


config.json:   0%|          | 0.00/5.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.13G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.58k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/851k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/217 [00:00<?, ?B/s]

In [16]:
from transformers import pipeline
import requests
from PIL import Image
from io import BytesIO

# Initialize the Pix2Struct image captioning pipeline
pix2struct_caption_pipeline = pipeline("image-to-text", model="google/pix2struct-textcaps-base")

# Updated function to generate a caption using Pix2Struct
def generate_caption_pix(image_url):
    try:
        response = requests.get(image_url)
        if response.status_code == 200:
            image = Image.open(BytesIO(response.content))
            result = pix2struct_caption_pipeline(image)
            return result[0]['generated_text'] if result else None
        else:
            return None
    except Exception as e:
        print(f"Error processing image {image_url}: {e}")
        return None

# Initialize lists for storing valid captions
valid_original_captions = []
valid_generated_captions = []

for item in subset:
    image_url = item['image_url']  # Adjust this key based on your dataset's structure
    original_caption = item['caption'].strip() if 'caption' in item and item['caption'].strip() else None

    # Generate a caption and proceed only if both original and generated captions are valid
    generated_caption = generate_caption_pix(image_url)
    if original_caption and generated_caption:
        print(f"Original Caption: {original_caption}")
        print(f"Generated Caption: {generated_caption}\n")

        valid_original_captions.append(original_caption)
        valid_generated_captions.append(generated_caption)
    else:
        # This discards any items with errors or missing original captions
        print(f"Discarding item due to error or missing original caption. Image URL: {image_url}")



Original Caption: a very typical bus station
Generated Caption: A man is standing in front of a large red car that says 9 on it.

Original Caption: sierra looked stunning in this top and this skirt while performing with person at their former university
Generated Caption: A man wearing a shirt that says Sierra Deaton's Closet.

Discarding item due to error or missing original caption. Image URL: https://media.gettyimages.com/photos/young-confused-girl-standing-in-front-of-a-wardrobe-picture-id511063329?s=612x612
Original Caption: interior design of modern living room with fireplace in a new house
Generated Caption: A sign advertises Shutterstock.com.

Original Caption: cybernetic scene isolated on white background .
Generated Caption: A phone that says shutterstock on it

Discarding item due to error or missing original caption. Image URL: https://media.gettyimages.com/photos/jayz-attends-the-chicago-bulls-vs-brooklyn-nets-playoff-game-at-the-picture-id167112882
Original Caption: the j

In [17]:
import nltk
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer

nltk.download('wordnet')

# Compute evaluation scores
tokenized_originals = [[oc.split()] for oc in valid_original_captions]
tokenized_generated = [gc.split() for gc in valid_generated_captions]

# BLEU score calculation
chencherry = SmoothingFunction()
bleu_score = corpus_bleu(tokenized_originals, tokenized_generated, smoothing_function=chencherry.method1)

# METEOR scores calculation with tokenized inputs
meteor_scores = [meteor_score([o.split()], g.split()) for o, g in zip(valid_original_captions, valid_generated_captions)]
avg_meteor = sum(meteor_scores) / len(meteor_scores)

# ROUGE scores calculation, including ROUGE-2
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = [scorer.score(o, g) for o, g in zip(valid_original_captions, valid_generated_captions)]
avg_rouge1 = sum(r['rouge1'].fmeasure for r in rouge_scores) / len(rouge_scores)
avg_rouge2 = sum(r['rouge2'].fmeasure for r in rouge_scores) / len(rouge_scores)
avg_rougeL = sum(r['rougeL'].fmeasure for r in rouge_scores) / len(rouge_scores)

print(f"BLEU Score: {bleu_score}")
print(f"Average METEOR Score: {avg_meteor}")
print(f"Average ROUGE-1 F Measure: {avg_rouge1}")
print(f"Average ROUGE-2 F Measure: {avg_rouge2}")
print(f"Average ROUGE-L F Measure: {avg_rougeL}")


BLEU Score: 0.0033929363106384456
Average METEOR Score: 0.038003845878252916
Average ROUGE-1 F Measure: 0.0768799827623357
Average ROUGE-2 F Measure: 0.0
Average ROUGE-L F Measure: 0.0768799827623357


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
